## Data Scraping, Standardization and Automation System for California Construction Projects

### Tools Used- 
1. Beautiful Soup (Web Scraping)
2. Open AI API (Standardization)
3. Openpyxl (Output Generation)

### Importing Neccessary Packages

In [1]:
import requests
from bs4 import BeautifulSoup
from langchain.llms import OpenAI
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import pandas as pd

### Web Scraping using Beautiful Soup

In [2]:
#Accepting the source url and writing the html data in a file
def fetchAndSaveToFile(url,path):
    r = requests.get(url)
    with open(path,"w") as f:
        f.write(r.text)


url = "https://projects.constructconnect.com/market-report/California-construction-projects?campaign=21222206374&group=158690373822&target=kwd-2297821187971&matchtype=e&creative=697443596315&device=c&se_kw=construction%20projects%20in%20california&utm_medium=ppc&utm_campaign=States&utm_term=construction%20projects%20in%20california&utm_source=adwords&hsa_ad=697443596315&hsa_kw=construction%20projects%20in%20california&hsa_net=adwords&hsa_tgt=kwd-2297821187971&hsa_grp=158690373822&hsa_src=g&hsa_ver=3&hsa_cam=21222206374&hsa_mt=e&hsa_acc=3324869874&gad_source=1&gclid=CjwKCAjwuJ2xBhA3EiwAMVjkVErjhjkbF6VTdQltFXD-nzJR5oWj2gXjO8SS5yo1v_AUK9KiGZtREhoC7VwQAvD_BwE"

In [3]:
fetchAndSaveToFile(url,"data/art2.html")

In [4]:
with open("data/art1.html","r") as f:
    html_doc = f.read()

In [5]:
#Beautiful Soup block to extract addresses
soup = BeautifulSoup(html_doc, 'html.parser')
locations = soup.findAll("a",class_="cc_child_menu_item")
p_texts = [p_tag.p.get_text(strip=True) for p_tag in locations]
print(p_texts)

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


### Data Standardization using OpenAI API

In [6]:
#Initializing the LLM
llm = OpenAI(api_key='provide_key', temperature=0.7)


In [7]:
#Creating a langchain prompt template to be fed to the LLM
outputs = []

prompt_template_name = PromptTemplate(
    input_variables = ['location'],
    template = "Provide me with a unique ID, the country name, country code, region name, region code, and geo location for {location}."
)

for location in p_texts:
    output = prompt_template_name.format(location=location)
    outputs.append(output)

print(outputs)

['Provide me with a unique ID, the country name, country code, region name, region code, and geo location for Alabama.', 'Provide me with a unique ID, the country name, country code, region name, region code, and geo location for Alaska.', 'Provide me with a unique ID, the country name, country code, region name, region code, and geo location for Arizona.', 'Provide me with a unique ID, the country name, country code, region name, region code, and geo location for Arkansas.', 'Provide me with a unique ID, the country name, country code, region name, region code, and geo location for California.', 'Provide me with a unique ID, the country name, country code, region name, region code, and geo location for Colorado.', 'Provide me with a unique ID, the country name, country code, region name, region code, and geo location for Connecticut.', 'Provide me with a unique ID, the country name, country code, region name, region code, and geo location for Delaware.', 'Provide me with a unique ID, 

In [8]:
#Obtaining the desired result from the LLM
output_text = []

for city in outputs:
    output_text.append(llm(city))
    
print(output_text)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


['\n\nID: AL001\nCountry Name: United States\nCountry Code: US\nRegion Name: Alabama\nRegion Code: AL\nGeo Location: 32.3182° N, 86.9023° W', '\n\nUnique ID: 1\nCountry name: United States\nCountry code: US\nRegion name: Alaska\nRegion code: AK\nGeo location: 64.2008° N, 149.4937° W', '\n\nID: AZ001\nCountry name: United States\nCountry code: US\nRegion name: Arizona\nRegion code: AZ\nGeo location: 34.0489° N, 111.0937° W', '\n\nID: US-AR\nCountry Name: United States\nCountry Code: US\nRegion Name: Arkansas\nRegion Code: AR\nGeo Location: 34.7465° N, 92.2896° W', '\n\nID: 12345\nCountry Name: United States\nCountry Code: US\nRegion Name: California\nRegion Code: CA\nGeo Location: 36.7783° N, 119.4179° W', '\n\nUnique ID:  1\n\nCountry name: United States\n\nCountry code: US\n\nRegion name: Colorado\n\nRegion code: CO\n\nGeo location: 39.5501° N, 105.7821° W', '\n\nUnique ID: 09c49549-1e1c-4078-a9d7-6c3629a17d3f\nCountry Name: United States\nCountry Code: US\nRegion Name: Connecticut\nR

### Preparing the output

In [ ]:
# Initialize lists to store data
ids = []
country_names = []
country_codes = []
region_names = []
region_codes = []
geo_locations = []

# Extract data from each record
for record in output_text:
    lines = record.split('\n')
    id_ = country_name = country_code = region_name = region_code = geo_location = None
    for line in lines:
        if 'Unique ID:' in line:
            id_ = line.split(': ')[1]
        elif 'Country Name:' in line:
            country_name = line.split(': ')[1]
        elif 'Country Code:' in line:
            country_code = line.split(': ')[1]
        elif 'Region Name:' in line:
            region_name = line.split(': ')[1]
        elif 'Region Code:' in line:
            region_code = line.split(': ')[1]
        elif 'Geo Location:' in line:
            geo_location = line.split(': ')[1]
    ids.append(id_)
    country_names.append(country_name)
    country_codes.append(country_code)
    region_names.append(region_name)
    region_codes.append(region_code)
    geo_locations.append(geo_location)

# Create a DataFrame using the extracted data
df = pd.DataFrame({
    'ID': ids,
    'Country Name': country_names,
    'Country Code': country_codes,
    'Region Name': region_names,
    'Region Code': region_codes,
    'Geo Location': geo_locations
})

# Define the file path for the Excel file
file_path = 'output_it3.xlsx'

# Save the DataFrame to Excel
df.to_excel(file_path, index=False)

print(f"Output saved to {file_path}")

Output saved to output_it3.xlsx
